# Po Valley Resilience & Rice Monitoring Framework
## Submission Overview

**Objective:**
We aim to collect data from disparate satellite sources, isolate specific crop windows, and extract distinctive crop characteristics to build a comprehensive crop monitoring framework.

### Definitions

**Table 1: Study Regions**

| Region | Location | Coordinates (BBox) | Purpose |
|---|---|---|---|
| **Region A** | Vercelli, Italy | `[8.34, 45.27, 8.40, 45.31]` | Primary focus area. Represents favorable, high-yield rice phenology. |
| **Region B** | Pavia, Italy | `[9.022, 45.161, 9.055, 45.182]` | Secondary focus area. Represents environmentally stressed rice conditions for comparative analysis. |

**Table 2: Sensor Characteristics & Objectives**

| Sensor | Source Catalog | Target Data Layer | Derived Characteristic / Objective |
|---|---|---|---|
| **Sentinel-1 (Radar)** | EOPF STAC | VV / VH Backscatter | Determine physical canopy structure, flooded field status, and moisture penetration. |
| **Sentinel-2 (Optical)** | Earth Search AWS | Red, NIR, SWIR Bands | Calculate NDVI (Vegetation Greenness), distinguishing crop health and growth cycles. |
| **Sentinel-3 (Thermal)** | EOPF STAC | Land Surface Temp (LST) | Assess canopy heat stress and micro-climate anomalies over the season. |

**Methodological Workflow:**
1. **Setup & Initialization:** Load essential geospatial libraries and initiate parallel processing clusters.
2. **Study Area Definition:** Establish precise geographic coordinates for the target regions.
3. **Data Discovery & STAC API Setup:** Connect to both EOPF and Earth Search catalogs for multi-sensor comparisons.
4. **Geospatial Visualization:** Validate geographic boundaries via interactive web mappings.
5. **Data Extraction & Zarr Cube Generation:** Generate optimized, cloud-native analysis-ready data structures containing crop health metrics (NDVI).
6. **Multi-Sensor Extension (S1 & S3):** Query radar and thermal data streams for structure and stress indicators.
7. **Time-Series Metric Extraction:** Compute and map the temporal progression of crop indicators across the growing season.
8. **Comprehensive Availability Coverage:** Review the statistical footprint and cross-sensor coverage available over the target sites.

---

### =============================================================
### Step 1: Setup & Initialization
Import essential analytical tools and create a localized Dask cluster for parallel processing.

**Expected Output:** Successful library instantiation, and initialization logs for the Dask cluster.
### =============================================================

In [ ]:
# Install missing dependencies for the notebook directly into the active kernel environment
!pip install -q pystac-client odc-stac folium xarray zarr s3fs dask fsspec seaborn geopandas

In [ ]:
import os
# Workaround for Anaconda PROJ version conflicts
os.environ.pop('PROJ_LIB', None)
os.environ.pop('PROJ_DATA', None)
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pystac_client

from pystac_client import Client
import shutil

import odc.stac

import fsspec
import os

from datetime import datetime
from pystac_client import CollectionSearch
from shapely import geometry
from distributed import LocalCluster

In [ ]:
cluster = LocalCluster(processes=False)
client = cluster.get_client()
cluster

In [ ]:
search = CollectionSearch(
    url="https://earth-search.aws.element84.com/v1/collections",
)
for collection_dict in search.collections_as_dicts():
    print(collection_dict["id"])

### =============================================================
### Step 2: Study Area Definition
Here we define the precise coordinates (Bounding Boxes) and the date calendar for our analysis. 

- **Timeframe:** April 1st, 2025 to December 1st, 2025 (Capturing the entire rice planting, growing, and harvesting season).
- **Cloud Filter:** We command the STAC catalog to only show us images that have less than 50% cloud cover.

**Expected Output:** Assignment of geographic coordinates and calendar dates to Python variables.
### =============================================================

In [ ]:
# Vercelli, Italy (Rice fields)
REGION_A = [8.34, 45.27, 8.40, 45.31]
DATE_RANGE_A = "2025-04-01/2025-12-01"

# Pavia, Italy (Rice fields)
REGION_B = [9.022, 45.161, 9.055, 45.182]
DATE_RANGE_B = "2025-04-01/2025-12-01"

CLOUD_COVER_LIMIT = 50  # percent

### =============================================================
### Step 3: STAC Catalog Comparison
STAC (SpatioTemporal Asset Catalog) is a standardized library for satellite imagery. Instead of downloading heavy files blindly, we use STAC APIs to "ask" the database what images are available over our study regions.

We compare two major catalogs to select the best data source:
1. **EOPF STAC:** Official European Space Agency (ESA) catalog.
2. **Earth Search AWS:** Amazon's global catalog holding Cloud-Optimized GeoTIFFs (COGs).

*The code block below will dynamically query both catalogs and generate a summary table comparing their coverage over Region A and Region B.*
### =============================================================

In [ ]:
# COMPARE BOTH STAC CATALOGS
# ---------------------------------------------------------
import pystac_client

REGION = REGION_A
DATES = DATE_RANGE_A

print("=" * 60)
print("COMPARISON: EOPF vs Earth Search AWS - REGION A (Vercelli)")
print("=" * 60)

# 1. EOPF STAC
print("\n1. EOPF STAC (https://stac.core.eopf.eodc.eu)")
print("-" * 60)
eopf_client = pystac_client.Client.open("https://stac.core.eopf.eodc.eu")
eopf_search = eopf_client.search(
    collections=["sentinel-2-l2a"],
    bbox=REGION,
    datetime=DATES,
    query={"eo:cloud_cover": {"lt": CLOUD_COVER_LIMIT}}
)
eopf_items = list(eopf_search.items())
print(f"✓ Total scenes found: {len(eopf_items)}")
print(f"✓ Band naming: B02_10m, B03_10m, B04_10m, B08_10m, SCL_20m")
print(f"✓ Location: European servers (EODC)")
print(f"✓ Format: Original ESA Sentinel-2 naming")

# 2. Earth Search AWS
print("\n2. Earth Search AWS (https://earth-search.aws.element84.com/v1)")
print("-" * 60)
aws_client = pystac_client.Client.open("https://earth-search.aws.element84.com/v1")
aws_search = aws_client.search(
    collections=["sentinel-2-l2a"],
    bbox=REGION,
    datetime=DATES,
    query={"eo:cloud_cover": {"lt": CLOUD_COVER_LIMIT}}
)
aws_items = list(aws_search.items())
print(f"✓ Total scenes found: {len(aws_items)}")
print(f"✓ Band naming: red, nir, blue, green, scl (lowercase aliases)")
print(f"✓ Location: AWS S3 (global)")
print(f"✓ Format: Cloud-optimized COGs with simplified names")

# RECOMMENDATION
print("\n" + "=" * 60)
print("RECOMMENDATION:")
print("=" * 60)
if len(aws_items) > len(eopf_items):
    print(f"🏆 Earth Search AWS has MORE scenes ({len(aws_items)} vs {len(eopf_items)}) on Region A")
    print("   → Better for comprehensive time series analysis")
    print("   → Faster global access via AWS")
    print("   → Simpler band naming (red, nir, scl)")
elif len(eopf_items) > len(aws_items):
    print(f"🏆 EOPF has MORE scenes ({len(eopf_items)} vs {len(aws_items)})")
    print("   → Count: Better for European data")
    print("   → Official ESA naming convention")
else:
    print(f"Both have the same number of scenes ({len(aws_items)})")
    print("   → Choose based on access speed and naming preference")


In [ ]:
# COMPARE BOTH STAC CATALOGS
# ---------------------------------------------------------
import pystac_client

REGION = REGION_B
DATES = DATE_RANGE_B

print("=" * 60)
print("COMPARISON: EOPF vs Earth Search AWS - REGION B (Pavia)")
print("=" * 60)

# 1. EOPF STAC
print("\n1. EOPF STAC (https://stac.core.eopf.eodc.eu)")
print("-" * 60)
eopf_client = pystac_client.Client.open("https://stac.core.eopf.eodc.eu")
eopf_search = eopf_client.search(
    collections=["sentinel-2-l2a"],
    bbox=REGION,
    datetime=DATES,
    query={"eo:cloud_cover": {"lt": CLOUD_COVER_LIMIT}}
)
eopf_items_b = list(eopf_search.items())
print(f"✓ Total scenes found: {len(eopf_items_b)}")
print(f"✓ Band naming: B02_10m, B03_10m, B04_10m, B08_10m, SCL_20m")
print(f"✓ Location: European servers (EODC)")
print(f"✓ Format: Original ESA Sentinel-2 naming")

# 2. Earth Search AWS
print("\n2. Earth Search AWS (https://earth-search.aws.element84.com/v1)")
print("-" * 60)
aws_client = pystac_client.Client.open("https://earth-search.aws.element84.com/v1")
aws_search = aws_client.search(
    collections=["sentinel-2-l2a"],
    bbox=REGION,
    datetime=DATES,
    query={"eo:cloud_cover": {"lt": CLOUD_COVER_LIMIT}}
)
aws_items_b = list(aws_search.items())
print(f"✓ Total scenes found: {len(aws_items_b)}")
print(f"✓ Band naming: red, nir, blue, green, scl (lowercase aliases)")
print(f"✓ Location: AWS S3 (global)")
print(f"✓ Format: Cloud-optimized COGs with simplified names")

# RECOMMENDATION
print("\n" + "=" * 60)
print("RECOMMENDATION:")
print("=" * 60)
if len(aws_items_b) > len(eopf_items_b):
    print(f"🏆 Earth Search AWS has MORE scenes ({len(aws_items_b)} vs {len(eopf_items_b)}) on Region B")
    print("   → Better for comprehensive time series analysis")
    print("   → Faster global access via AWS")
    print("   → Simpler band naming (red, nir, scl)")
elif len(eopf_items_b) > len(aws_items_b):
    print(f"🏆 EOPF has MORE scenes ({len(eopf_items_b)} vs {len(aws_items_b)})")
    print("   → Better for European data")
    print("   → Official ESA naming convention")
else:
    print(f"Both have the same number of scenes ({len(aws_items_b)})")
    print("   → Choose based on access speed and naming preference")


In [ ]:
import pandas as pd

# Create a dynamic comparison table based on the items found above
summary_data = {
    "Feature": [
        "Scenes Found (Region A)", 
        "Scenes Found (Region B)", 
        "Band Naming", 
        "Storage Location", 
        "Format"
    ],
    "EOPF STAC": [
        len(eopf_items),  # Dynamically pull from Region A query
        len(eopf_items_b),
        "Official ESA (e.g., B02_10m)", 
        "European Servers (EODC)", 
        "Standard"
    ],
    "Earth Search AWS": [
        len(aws_items),   # Dynamically pull from Region A query
        len(aws_items_b),
        "Simplified Aliases (e.g., blue)", 
        "Global AWS S3", 
        "Cloud-Optimized (Faster loading)"
    ]
}

df_comparison = pd.DataFrame(summary_data)

print("\n============================================================")
print("STAC CATALOG COMPARISON SUMMARY")
print("============================================================")
display(df_comparison)

print("\nConclusion: Earth Search AWS is selected due to significantly higher valid scene counts and cloud-optimized formatting.")

### Monthly Coverage Check
Before proceeding, we tally the exact number of valid Sentinel-2 scenes available per month across the 2025 growing season (April -> November) from our chosen catalog (Earth Search AWS). 

This confirms we have consistent satellite coverage throughout the entire crop cycle for both **Region A** and **Region B**, ensuring our final time-series charts will not have massive gaps.

In [ ]:
import pystac_client
import pandas as pd

# STAC API Setup
STAC_URL = "https://earth-search.aws.element84.com/v1"

# Corrected Collection Variables
COLLECTIONS = {
    "Sentinel-2 (L2A)": "sentinel-2-l2a"
}

# Connect to Client once
catalog = pystac_client.Client.open(STAC_URL)

for friendly_name, collection_id in COLLECTIONS.items():
    print(f"\n--- Searching {friendly_name} from Earth Search AWS [Region A] ---")
    
    search = catalog.search(
        collections=[collection_id],
        bbox=REGION_A,
        datetime=DATE_RANGE_A,
        query={"eo:cloud_cover": {"lt": CLOUD_COVER_LIMIT}}
    )
    
    items = list(search.items())
    print(f"Total items found: {len(items)}")
    
    if items:
        # Extract datetimes from items
        # STAC items always have a .datetime property (UTC)
        dates = [item.datetime for item in items]
        
        # Create a Pandas DataFrame for easy grouping
        df = pd.DataFrame(dates, columns=['date'])
        
        # Convert to datetime objects if not already
        df['date'] = pd.to_datetime(df['date'])
        
        # Group by Year-Month and count
        # 'M' stands for Month end frequency
        monthly_counts_a = df.groupby(df['date'].dt.strftime('%m-%Y')).size()
        
        print("Items per month:")
        print(monthly_counts_a)
    else:
        print("No items found in this range.")

In [ ]:
# Connect to Client once
catalog = pystac_client.Client.open(STAC_URL)

for friendly_name, collection_id in COLLECTIONS.items():
    print(f"\n--- Searching {friendly_name} from Earth Search AWS [Region B] ---")
    
    search = catalog.search(
        collections=[collection_id],
        bbox=REGION_B,
        datetime=DATE_RANGE_B,
        query={"eo:cloud_cover": {"lt": CLOUD_COVER_LIMIT}}
    )
    
    items = list(search.items())
    print(f"Total items found: {len(items)}")
    
    if items:
        # Extract datetimes from items
        # STAC items always have a .datetime property (UTC)
        dates = [item.datetime for item in items]
        
        # Create a Pandas DataFrame for easy grouping
        df = pd.DataFrame(dates, columns=['date'])
        
        # Convert to datetime objects if not already
        df['date'] = pd.to_datetime(df['date'])
        
        # Group by Year-Month and count
        # 'M' stands for Month end frequency
        monthly_counts_b = df.groupby(df['date'].dt.strftime('%m-%Y')).size()
        
        print("Items per month:")
        print(monthly_counts_b)
    else:
        print("No items found in this range.")

In [ ]:
import pandas as pd

df_summary = pd.DataFrame({
    'Region A': monthly_counts_a,
    'Region B': monthly_counts_b
})

df_summary.index.name = 'MMYY'
df_summary = df_summary.fillna(0).astype(int)

print("\n============================================================")
print("Summary of Sentinel-2 (L2A) from Earth Search AWS in [Region A] and [Region B]")
print("============================================================")
display(df_summary)

### =============================================================
### Step 4: Geospatial Visualization
Before we start crunching massive amounts of data, it's crucial to visually verify that our Bounding Boxes (`REGION_A` and `REGION_B`) actually cover the agricultural fields we want to analyze. 

We use **Folium**, an interactive mapping library, to overlay our coordinates onto a real-world map. 
*Note: You can zoom in and drag the map around.*

**Expected Output:** Interactive Folium widgets showing red rectangles over the target Italian farming regions.
### =============================================================

In [ ]:
import folium

min_lon, min_lat, max_lon, max_lat = REGION_A
center = [(min_lat + max_lat)/2, (min_lon + max_lon)/2]

m = folium.Map(location=center, zoom_start=14, tiles='CartoDB positron')
folium.Rectangle(
    bounds=[(min_lat, min_lon), (max_lat, max_lon)],
    color="red", fill=True, fill_opacity=0.15
).add_to(m)

m


In [ ]:
import folium

min_lon, min_lat, max_lon, max_lat = REGION_B
center = [(min_lat + max_lat)/2, (min_lon + max_lon)/2]

m = folium.Map(location=center, zoom_start=14, tiles='CartoDB positron')
folium.Rectangle(
    bounds=[(min_lat, min_lon), (max_lat, max_lon)],
    color="red", fill=True, fill_opacity=0.15
).add_to(m)

m


### =============================================================
### Step 5: Data Extraction & Zarr Cube Generation (From Earch Search AWS)

This is the most critical step in the notebook. Now that we know Earth Search AWS has the best optical coverage, we will:

1. **Lazy Load (odc.stac):** We tell the computer to "peek" at the satellite images (Red, Near-Infrared, and Scene Classification mask) without downloading the heavy files directly into your computer's RAM. 
2. **Masking (Cloud Removal):** Satellite images often have clouds or shadows blocking the farms. We use the Scene Classification band (`scl`) to strip away clouds, keeping only clear vegetation, bare soil, and water.
3. **Calculate NDVI (Crop Health):** NDVI (Normalized Difference Vegetation Index) is the standard formula for plant health. It mathematically compares Near-Infrared light (which healthy leaves reflect) with Red light (which plants absorb). *Higher values = greener, healthier crops.*
4. **Save to Zarr:** We take this massive, multi-dimensional block of calculated NDVI data (X, Y, and Time) and save it to your hard drive as a `.zarr` folder. Zarr is a highly compressed, cloud-optimized format that makes plotting the data lightning fast later.

**Expected Output:** The code will process hundreds of Gigabytes of remote data, reduce it to just NDVI, and save a persistent Zarr directory for both Region A and Region B.
### =============================================================

In [ ]:
# INSPECT AVAILABLE BANDS
# ---------------------------------------------------------
print("Checking available bands/assets in the first item...")
if len(items) > 0:
    first_item = items[0]
    print("\nAvailable assets:")
    for asset_key in first_item.assets.keys():
        print(f"  - {asset_key}")


### Step 6: NVDI: Visualizing the Growth Cycle

Based on the processed Zarr data, we can now map the true vegetative growth cycle of the region. Our methodology strictly separates verified sensor data from mathematical estimation:

1. **Primary Data Source:** We use the local `.zarr` files generated in the previous step, built exclusively from the high-frequency Earth Search AWS catalog.
2. **Cloud Processing (Masking):** The Zarr engine already parsed the `scl` (Scene Classification Layer) band. All pixels identified as cloud or cloud shadow have been intentionally dropped (marked as `NaN`) to prevent contaminated pixels from ruining the vegetative health score.
3. **Filtering and Interpolation:** 
    - **Ground Truth (Green Dots):** The remaining, verified clear-sky averages are plotted as distinct points. If a satellite pass was completely obscured by clouds over the Region bounding box, no dot is drawn.
    - **Continuous Estimation (Orange Line):** We perform a linear interpolation across the missing (cloudy) dates to provide a visual approximation of the continuous crop phenology.
    - **Data Integrity:** The final tabular output permanently drops completely obscured records, presenting only the valid ground truth alongside our interpolation.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

ds_local = xr.open_dataset(zarr_path, engine='zarr', decode_cf=True)

# Calculate Spatial mean for time series (this contains NaNs where clouds were masked)
ts = ds_local.NDVI.mean(dim=["x", "y"], skipna=True)

# 1. Convert to a Pandas Series for easy interpolation
df_ts = ts.to_pandas()

# 2. Interpolate the missing NaN values (linear interpolation connects the dots)
df_ts_interpolated = df_ts.interpolate(method='linear')

# 3. Plotting
plt.figure(figsize=(12, 6))

# Plot the interpolated data as a continuous orange line
df_ts_interpolated.plot(kind='line', color='orange', linestyle='--', label='Interpolated Data')

# Plot the original data as green dots (no line, just the valid points)
df_ts.plot(style='o', color='green', label='Original Data (Cloud-free)')

plt.title("Phenology (growth cycle) of the rice fields (April to December)")
plt.ylabel("NDVI")
plt.xlabel("Date")
plt.grid(True)
plt.legend()
plt.show()

# 4. Display a table of the results
print("\n============================================================")
print(f"NDVI Time Series Values ({zarr_path})")
print("============================================================")

# Create a clean DataFrame for display
df_results = pd.DataFrame({
    'Original NDVI': df_ts.round(3),
    'Interpolated NDVI': df_ts_interpolated.round(3)
})

# Filter out rows where both are NaN just to keep table clean (optional, but good practice)
df_results = df_results.dropna(subset=['Interpolated NDVI'])

# Show as an interactive table
display(df_results)


In [ ]:
import os
# Workaround for Anaconda PROJ version conflicts
os.environ.pop('PROJ_LIB', None)
os.environ.pop('PROJ_DATA', None)
import os
import shutil
from pystac_client import Client
import odc.stac
import xarray as xr
import matplotlib.pyplot as plt

STAC_URL = "https://earth-search.aws.element84.com/v1"
COLLECTION = "sentinel-2-l2a"

# pavia, Italy (Rice fields)
BBOX = [9.022, 45.161, 9.055, 45.182]
DATES = "2025-04-01/2025-12-01"

print("Searching STAC catalog...")
client = Client.open(STAC_URL)

search = client.search(
    collections=[COLLECTION],
    bbox=BBOX,
    datetime=DATES,
    # Relaxed cloud cover slightly to ensure we get hits, 
    # we handle masking later anyway.
    query={"eo:cloud_cover": {"lt": 50}} 
)

items = list(search.item_collection())
print(f"Found {len(items)} scenes.")

# --- SAFETY CHECK ---
if len(items) == 0:
    raise ValueError("No scenes found! Try increasing the date range or removing the cloud filter.")

# 2. LAZY LOAD DATA
# ---------------------------------------------------------
print("Configuring lazy load...")
ds = odc.stac.load(
    items,
    bands=["red", "nir", "scl"], # Note: Earth Search uses lowercase aliases often, but B04/B08 work too
    bbox=BBOX,
    resolution=10,
    chunks={"x": 1024, "y": 1024, "time": 1},
    groupby="solar_day",
    crs="EPSG:32632"
)

# 3. MASKING & NDVI
# ---------------------------------------------------------
# Earth Search SCL: 0=No Data, 1=Saturated, 3=Shadow, 8-10=Clouds, 4=Veg, 5=Bare, 6=Water
valid_mask = (ds.scl == 4) | (ds.scl == 5) | (ds.scl == 6) | (ds.scl == 7)

# Scale factors: Earth Search data is usually uint16 (0-10000)
red = ds.red.where(valid_mask) / 10000.0
nir = ds.nir.where(valid_mask) / 10000.0

ndvi = (nir - red) / (nir + red)
ndvi = ndvi.rename("NDVI")

# 4. SAVE TO ZARR
# ---------------------------------------------------------
zarr_path = "rice_ndvi_cube_region_b.zarr"
if os.path.exists(zarr_path):
    shutil.rmtree(zarr_path)

print(f"Computing and saving to {zarr_path}...")
# This is the heavy lifting step
d = ndvi.to_dataset()
# Forcefully remove all metadata to bypass xarray/zarr Float32 bug
d.attrs.clear()
for var in d.variables:
    d[var].attrs.clear()
    d[var].encoding = {'_FillValue': None}
# Save using safely configured engine
d.to_zarr(zarr_path, mode="w", safe_chunks=False)
print("Zarr write complete.")

# 5. PLOT (Using the saved Zarr)
# ---------------------------------------------------------
ds_local = xr.open_dataset(zarr_path, engine='zarr', decode_cf=False, mask_and_scale=False)

# Spatial mean for time series
ts = ds_local.NDVI.mean(dim=["x", "y"], skipna=True)

plt.figure(figsize=(12, 6))
ts.plot.line(marker="o", color="green")
plt.title("NDVI Time Series (Rice Detection)")
plt.ylabel("NDVI")
plt.grid(True)
plt.show()

In [ ]:

# Note: Continuous lines use linear interpolation to bridge gaps caused by cloud-cover days.
# Load both regions' NDVI data from Zarr
import pandas as pd

# Region A data
ds_region_a = xr.open_dataset("rice_ndvi_cube_region_a.zarr", engine='zarr', decode_cf=False, mask_and_scale=False)
ts_a = ds_region_a.NDVI.mean(dim=["x", "y"], skipna=True)
data_region_a = pd.DataFrame({
    'date': ts_a.time.values,
    'ndvi_mean': ts_a.values
})

# Region B data
ds_region_b = xr.open_dataset("rice_ndvi_cube_region_b.zarr", engine='zarr', decode_cf=False, mask_and_scale=False)
ts_b = ds_region_b.NDVI.mean(dim=["x", "y"], skipna=True)
data_region_b = pd.DataFrame({
    'date': ts_b.time.values,
    'ndvi_mean': ts_b.values
})

fig, ax = plt.subplots(figsize=(12, 6))

# Plot Region A
ax.plot(data_region_a['date'], data_region_a['ndvi_mean'].interpolate(method='linear'), 
        marker='o', linewidth=2.5, markersize=8, 
        label='Region A (Favorable)', color='#2ecc71')

# Plot Region B
ax.plot(data_region_b['date'], data_region_b['ndvi_mean'].interpolate(method='linear'), 
        marker='s', linewidth=2.5, markersize=8, 
        label='Region B (Stress)', color='#e74c3c')

# Add reference line for critical NDVI threshold
ax.axhline(y=0.6, color='gray', linestyle='--', linewidth=1.5, alpha=0.6, 
           label='Optimal NDVI threshold')

ax.set_xlabel('Month (2025)', fontsize=12, fontweight='bold')
ax.set_ylabel('NDVI (Normalized Difference Vegetation Index)', fontsize=12, fontweight='bold')
ax.set_title('Crop Vegetation Dynamics: Region A vs Region B', fontsize=14, fontweight='bold', pad=15)
ax.legend(loc='upper left', fontsize=11, framealpha=0.9)
ax.grid(True, alpha=0.3)
ax.set_ylim(0.3, 0.85)

plt.tight_layout()
plt.show()

print("\nKey Statistics:")
print(f"  Region A - Peak NDVI: {data_region_a['ndvi_mean'].interpolate(method='linear').max():.3f}")
print(f"  Region B - Peak NDVI: {data_region_b['ndvi_mean'].interpolate(method='linear').max():.3f}")
print(f"  NDVI Difference: {(data_region_a['ndvi_mean'].interpolate(method='linear').max() - data_region_b['ndvi_mean'].interpolate(method='linear').max()):.3f} ({((data_region_a['ndvi_mean'].interpolate(method='linear').max() - data_region_b['ndvi_mean'].interpolate(method='linear').max()) / data_region_b['ndvi_mean'].interpolate(method='linear').max() * 100):.1f}% higher in Region A)")


### =============================================================
### Step 7: VV Backscatter: Multi-Sensor Extension (Sentinel-1 Radar)

Optical sensors (like Sentinel-2 above) are great for seeing "greenness", but they are completely blind when it's cloudy. 

Here we bring in **Sentinel-1**, which uses **Radar (SAR)**. Radar shoots microwaves down to Earth and measures how they bounce back (backscatter). Microwaves pierce right through clouds and rain! 

**Why use Radar for Rice?**
Rice is grown in flooded fields. 
- When fields are flooded (water), the radar bounces away like a mirror (Low VV Backscatter).
- When the rice grows tall out of the water, it scatters the radar effectively (High VV Backscatter).
By looking at the radar timeline, we can actually "see" the exact week the farmers flooded their fields and when the crop canopy closed over the water!

**Expected Output:** Statistical processing of radar backscatter over our study areas.
### =============================================================

In [ ]:
import pystac_client
import pandas as pd
import matplotlib.pyplot as plt

STAC_URL = "https://stac.core.eopf.eodc.eu"
COLLECTION_ID = "sentinel-1-l1-grd"
DATE_RANGE = "2020-04-01/2025-12-01"

REGIONS = {
    "REGION_A-italy": [8.34, 45.27, 8.40, 45.31],
    "REGION_B-spain": [0.55, 40.58, 0.90, 40.83],
}

catalog = pystac_client.Client.open(STAC_URL)

def monthly_counts_for_bbox(bbox, date_range):
    search = catalog.search(
        collections=[COLLECTION_ID],
        bbox=bbox,
        datetime=date_range
    )
    items = list(search.items())
    if not items:
        return items, pd.Series(dtype=int)

    dates = [it.datetime for it in items]
    df = pd.DataFrame({"date": pd.to_datetime(dates, utc=True)})
    # group by month (PeriodIndex), then convert to Timestamp later for plotting/merging
    monthly = df.groupby(df["date"].dt.to_period("M")).size().sort_index()
    return items, monthly

all_monthlies = {}
all_items = {}

print("--- Searching Sentinel-1 (GRD) on EOPF STAC ---")
for name, bbox in REGIONS.items():
    print(f"\n[{name}] bbox={bbox}  datetime={DATE_RANGE}")
    items, monthly = monthly_counts_for_bbox(bbox, DATE_RANGE)

    all_items[name] = items
    all_monthlies[name] = monthly

    print(f"Total items found: {len(items)}")
    if len(items) == 0:
        print("No items found for this region.")
    else:
        print("Items per month (head):")
        print(monthly.head(12))

# ---- combine into one table aligned by month ----
# Convert each Series' PeriodIndex to TimestampIndex so we can align across regions
aligned = []
for name, s in all_monthlies.items():
    if len(s) == 0:
        aligned.append(pd.Series(name=name, dtype=int))
    else:
        s2 = s.copy()
        s2.index = s2.index.to_timestamp()  # Period -> Timestamp
        s2.name = name
        aligned.append(s2)

counts_df = pd.concat(aligned, axis=1).fillna(0).astype(int)
counts_df = counts_df.sort_index()

print("\n--- Combined monthly counts (tail) ---")
print(counts_df.tail(12))

# ---- one bar chart for both regions ----
plt.figure(figsize=(14, 6))
counts_df.plot(kind="bar", width=0.9)  # grouped bars per month for both regions
plt.title("Sentinel-1 GRD availability per month (Italy vs Spain)")
plt.xlabel("Month")
plt.ylabel("Number of scenes")
plt.tight_layout()
plt.show()

# ---- inspect assets (one sample per region) ----
for name, items in all_items.items():
    if not items:
        continue

    sample = items[0]
    print(f"\n===== Sample inspection: {name} =====")
    print("Sample item id:", sample.id)
    print("Sample item datetime (UTC):", sample.datetime)

    print("\nAsset keys in sample item:")
    for k, a in sample.assets.items():
        mt = a.media_type if a.media_type else "None"
        href = a.href if a.href else ""
        print(f" - {k:20s} | type={mt} | href={href[:120]}...")

    zarr_like = [
        (k, a.href) for k, a in sample.assets.items()
        if ((a.media_type and "zarr" in a.media_type.lower()) or (a.href and a.href.lower().endswith(".zarr")))
    ]
    print("\nZarr-like assets detected:" if zarr_like else "\nNo obvious zarr-like assets detected in sample.")
    for k, href in zarr_like:
        print(f" - {k}: {href}")

In [ ]:
import pystac_client
import pandas as pd
import matplotlib.pyplot as plt

STAC_URL = "https://stac.core.eopf.eodc.eu"
COLLECTION_ID = "sentinel-3-slstr-l2-lst"
DATE_RANGE = "2020-04-01/2025-12-01"

REGIONS = {
    "REGION_A-italy": [8.34, 45.27, 8.40, 45.31],
    "REGION_B-spain": [0.55, 40.58, 0.90, 40.83],
}

catalog = pystac_client.Client.open(STAC_URL)

def monthly_counts_for_bbox(bbox, date_range):
    search = catalog.search(
        collections=[COLLECTION_ID],
        bbox=bbox,
        datetime=date_range
    )
    items = list(search.items())
    if not items:
        return items, pd.Series(dtype=int)

    dates = [it.datetime for it in items]
    df = pd.DataFrame({"date": pd.to_datetime(dates, utc=True)})
    # group by month (PeriodIndex), then convert to Timestamp later for plotting/merging
    monthly = df.groupby(df["date"].dt.to_period("M")).size().sort_index()
    return items, monthly

all_monthlies = {}
all_items = {}

print("--- Searching Sentinel-3 (GRD) on EOPF STAC ---")
for name, bbox in REGIONS.items():
    print(f"\n[{name}] bbox={bbox}  datetime={DATE_RANGE}")
    items, monthly = monthly_counts_for_bbox(bbox, DATE_RANGE)

    all_items[name] = items
    all_monthlies[name] = monthly

    print(f"Total items found: {len(items)}")
    if len(items) == 0:
        print("No items found for this region.")
    else:
        print("Items per month (head):")
        print(monthly.head(12))

# ---- combine into one table aligned by month ----
# Convert each Series' PeriodIndex to TimestampIndex so we can align across regions
aligned = []
for name, s in all_monthlies.items():
    if len(s) == 0:
        aligned.append(pd.Series(name=name, dtype=int))
    else:
        s2 = s.copy()
        s2.index = s2.index.to_timestamp()  # Period -> Timestamp
        s2.name = name
        aligned.append(s2)

counts_df = pd.concat(aligned, axis=1).fillna(0).astype(int)
counts_df = counts_df.sort_index()

print("\n--- Combined monthly counts (tail) ---")
print(counts_df.tail(12))

# ---- one bar chart for both regions ----
plt.figure(figsize=(14, 6))
counts_df.plot(kind="bar", width=0.9)  # grouped bars per month for both regions
plt.title("Sentinel-3 LST availability per month (Italy vs Spain)")
plt.xlabel("Month")
plt.ylabel("Number of scenes")
plt.tight_layout()
plt.show()

# ---- inspect assets (one sample per region) ----
for name, items in all_items.items():
    if not items:
        continue

    sample = items[0]
    print(f"\n===== Sample inspection: {name} =====")
    print("Sample item id:", sample.id)
    print("Sample item datetime (UTC):", sample.datetime)

    print("\nAsset keys in sample item:")
    for k, a in sample.assets.items():
        mt = a.media_type if a.media_type else "None"
        href = a.href if a.href else ""
        print(f" - {k:20s} | type={mt} | href={href[:120]}...")

    zarr_like = [
        (k, a.href) for k, a in sample.assets.items()
        if ((a.media_type and "zarr" in a.media_type.lower()) or (a.href and a.href.lower().endswith(".zarr")))
    ]
    print("\nZarr-like assets detected:" if zarr_like else "\nNo obvious zarr-like assets detected in sample.")
    for k, href in zarr_like:
        print(f" - {k}: {href}")

### =============================================================
### Step 7: Time-Series Metric Extraction (The Results)

Now we load the `Zarr` data cubes we saved in Step 5 and plot them on a timeline. By putting Region A (Favorable) and Region B (Stressed) on the same graph, we can visually compare their growing seasons. 

*Note: You might see gaps or straight lines connecting dots. Because we removed clouds in Step 5, some weeks have no optical data. The code automatically connects the dots (interpolates) to show the general trend.*

**Expected Output:** Time-series line charts showing the phenological (growth) cycle of the rice crops across the year.
### =============================================================

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pystac_client import Client

# Use the regions from earlier
REGION_A_BBOX = [8.34, 45.27, 8.40, 45.31]  # Vercelli, Italy
REGION_B_BBOX = [9.022, 45.161, 9.055, 45.182]  # Pavia, Italy

# Date range for Sentinel-1 data
S1_DATE_RANGE = "2024-04-01/2025-12-01"

STAC_URL = "https://stac.core.eopf.eodc.eu"
COLLECTION_ID = "sentinel-1-l1-grd"

print("Fetching Sentinel-1 GRD data for Region A and Region B...")
print("=" * 60)

In [ ]:
def get_s1_vv_mean_odc(bbox, date_range, region_name):
    """
    Fetch Sentinel-1 GRD data using odc.stac and extract mean VV backscatter values
    """
    catalog = Client.open(STAC_URL)
    
    search = catalog.search(
        collections=[COLLECTION_ID],
        bbox=bbox,
        datetime=date_range
    )
    
    items = list(search.items())
    print(f"\n[{region_name}]")
    print(f"  Found {len(items)} Sentinel-1 scenes")
    
    if len(items) == 0:
        print(f"  No data found for {region_name}")
        return None
    
    # Check assets in first item
    sample = items[0]
    print(f"  Available assets: {list(sample.assets.keys())}")
    
    try:
        # Try using odc.stac to load the data
        print(f"  Attempting to load VV band using odc.stac...")
        ds = odc.stac.load(
            items,
            bands=["vv"],
            bbox=bbox,
            resolution=10,  # 10m resolution for Sentinel-1
            chunks={"x": 512, "y": 512, "time": 1},
            groupby="solar_day"
        )
        
        print(f"  Successfully loaded dataset")
        print(f"  Dataset variables: {list(ds.data_vars)}")
        print(f"  Dataset shape: {ds.dims}")
        
        # Extract VV data and compute spatial mean for each time step
        vv_data = ds.vv.mean(dim=["x", "y"], skipna=True).compute()
        
        # Create DataFrame
        df = pd.DataFrame({
            'date': vv_data.time.values,
            'vv_mean': vv_data.values
        })
        df = df.sort_values('date')
        
        # Remove NaN values
        df = df.dropna()
        
        print(f"  Successfully processed {len(df)} scenes")
        if len(df) > 0:
            print(f"  Mean VV backscatter: {df['vv_mean'].mean():.2f}")
        
        return df
        
    except Exception as e:
        print(f"  Error loading with odc.stac: {str(e)[:200]}")
        return None

# Process both regions
print("\nProcessing Region A (Vercelli, Italy)...")
df_region_a = get_s1_vv_mean_odc(REGION_A_BBOX, S1_DATE_RANGE, "Region A")

print("\nProcessing Region B (Pavia, Italy)...")
df_region_b = get_s1_vv_mean_odc(REGION_B_BBOX, S1_DATE_RANGE, "Region B")

In [ ]:
# Plot the VV backscatter time series for both regions
if df_region_a is not None and df_region_b is not None:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Plot Region A
    ax.plot(df_region_a['date'], df_region_a['vv_mean'], 
            marker='o', linewidth=2.5, markersize=8,
            label='Region A (Vercelli, Italy)', color='#3498db')
    
    # Plot Region B
    ax.plot(df_region_b['date'], df_region_b['vv_mean'],
            marker='s', linewidth=2.5, markersize=8,
            label='Region B (Pavia, Italy)', color='#e67e22')
    
    ax.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax.set_ylabel('VV Backscatter (dB)', fontsize=12, fontweight='bold')
    ax.set_title('Sentinel-1 VV Backscatter: Region A vs Region B', 
                 fontsize=14, fontweight='bold', pad=15)
    ax.legend(loc='best', fontsize=11, framealpha=0.9)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("\n" + "=" * 60)
    print("VV Backscatter Statistics:")
    print("=" * 60)
    print(f"\nRegion A (Vercelli, Italy):")
    print(f"  Mean VV: {df_region_a['vv_mean'].mean():.2f} dB")
    print(f"  Std Dev: {df_region_a['vv_mean'].std():.2f} dB")
    print(f"  Min VV: {df_region_a['vv_mean'].min():.2f} dB")
    print(f"  Max VV: {df_region_a['vv_mean'].max():.2f} dB")
    
    print(f"\nRegion B (Pavia, Italy):")
    print(f"  Mean VV: {df_region_b['vv_mean'].mean():.2f} dB")
    print(f"  Std Dev: {df_region_b['vv_mean'].std():.2f} dB")
    print(f"  Min VV: {df_region_b['vv_mean'].min():.2f} dB")
    print(f"  Max VV: {df_region_b['vv_mean'].max():.2f} dB")
    
    print(f"\nDifference (Region A - Region B):")
    print(f"  Mean difference: {(df_region_a['vv_mean'].mean() - df_region_b['vv_mean'].mean()):.2f} dB")
else:
    print("\n⚠️ Could not process VV data for one or both regions")

### Step 8: Comprehensive Availability Coverage
Compute absolute data coverage counts across sensors. This produces graphical analytics outlining the feasibility of a multi-sensor yield prediction model.

**Expected Output:** Deep heatmap visualizers, comparative mission bar-log charts, and terminal output describing raw STAC item saturation.

In [ ]:
import pystac_client
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

# Set up the STAC catalogs
EOPF_STAC_URL = "https://stac.core.eopf.eodc.eu"
AWS_STAC_URL = "https://earth-search.aws.element84.com/v1"

# Collections to query
COLLECTIONS = {
    "Sentinel-1 GRD": ("sentinel-1-l1-grd", EOPF_STAC_URL),
    "Sentinel-2 L2A": ("sentinel-2-l2a", AWS_STAC_URL),
    "Sentinel-3 LST": ("sentinel-3-slstr-l2-lst", EOPF_STAC_URL)
}

# Regions
REGIONS = {
    "Region A (Vercelli, Italy)": [8.34, 45.27, 8.40, 45.31],
    "Region B (Pavia, Italy)": [9.022, 45.161, 9.055, 45.182]
}

# Date range for comprehensive analysis
DATE_RANGE = "2024-01-01/2025-12-31"

print("Fetching data availability for all Sentinel missions...")
print("=" * 70)

In [ ]:
def get_monthly_counts(collection_name, collection_id, stac_url, bbox, date_range):
    """
    Get monthly scene counts for a specific collection and region
    """
    try:
        catalog = pystac_client.Client.open(stac_url)
        
        # Add cloud cover filter for Sentinel-2
        if "sentinel-2" in collection_id.lower():
            search = catalog.search(
                collections=[collection_id],
                bbox=bbox,
                datetime=date_range,
                query={"eo:cloud_cover": {"lt": 50}}
            )
        else:
            search = catalog.search(
                collections=[collection_id],
                bbox=bbox,
                datetime=date_range
            )
        
        items = list(search.items())
        
        if not items:
            return pd.Series(dtype=int)
        
        # Extract dates
        dates = [item.datetime for item in items]
        df = pd.DataFrame({"date": pd.to_datetime(dates, utc=True)})
        
        # Group by year-month
        monthly = df.groupby(df["date"].dt.to_period("M")).size().sort_index()
        
        return monthly
    
    except Exception as e:
        print(f"  Error fetching {collection_name}: {str(e)[:100]}")
        return pd.Series(dtype=int)

# Collect data for all combinations
all_data = {}

for region_name, bbox in REGIONS.items():
    print(f"\n{region_name}")
    print("-" * 70)
    
    region_data = {}
    
    for collection_name, (collection_id, stac_url) in COLLECTIONS.items():
        print(f"  Fetching {collection_name}...", end=" ")
        
        monthly = get_monthly_counts(collection_name, collection_id, stac_url, bbox, DATE_RANGE)
        
        if len(monthly) > 0:
            print(f"✓ {len(monthly)} months with data, {monthly.sum()} total scenes")
            region_data[collection_name] = monthly
        else:
            print("✗ No data found")
            region_data[collection_name] = pd.Series(dtype=int)
    
    all_data[region_name] = region_data

print("\n" + "=" * 70)
print("Data collection complete!")

In [ ]:
# Create comprehensive heatmap visualization
fig, axes = plt.subplots(2, 1, figsize=(16, 12))

for idx, (region_name, region_data) in enumerate(all_data.items()):
    ax = axes[idx]
    
    # Prepare data for heatmap
    # Get all unique months across all collections
    all_months = set()
    for collection_name, monthly in region_data.items():
        if len(monthly) > 0:
            all_months.update(monthly.index)
    
    if not all_months:
        ax.text(0.5, 0.5, f'No data available for {region_name}', 
                ha='center', va='center', fontsize=14)
        ax.set_title(region_name, fontsize=14, fontweight='bold', pad=15)
        continue
    
    # Sort months
    all_months = sorted(list(all_months))
    
    # Create matrix for heatmap
    heatmap_data = []
    collection_names = []
    
    for collection_name in ["Sentinel-1 GRD", "Sentinel-2 L2A", "Sentinel-3 LST"]:
        if collection_name in region_data:
            monthly = region_data[collection_name]
            row = []
            for month in all_months:
                if month in monthly.index:
                    row.append(monthly[month])
                else:
                    row.append(0)
            heatmap_data.append(row)
            collection_names.append(collection_name)
    
    # Convert to numpy array
    heatmap_array = np.array(heatmap_data)
    
    # Create heatmap
    sns.heatmap(heatmap_array, 
                ax=ax,
                cmap='YlOrRd',
                annot=True,
                fmt='d',
                cbar_kws={'label': 'Number of Scenes'},
                linewidths=0.5,
                linecolor='gray')
    
    # Set labels
    ax.set_yticklabels(collection_names, rotation=0, fontsize=11)
    ax.set_xticklabels([str(m) for m in all_months], rotation=45, ha='right', fontsize=9)
    ax.set_xlabel('Month', fontsize=12, fontweight='bold')
    ax.set_ylabel('Sentinel Mission', fontsize=12, fontweight='bold')
    ax.set_title(f'{region_name} - Data Availability Heatmap', 
                 fontsize=14, fontweight='bold', pad=15)

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("SUMMARY STATISTICS")
print("=" * 70)

In [ ]:
# Print detailed statistics
for region_name, region_data in all_data.items():
    print(f"\n{region_name}")
    print("-" * 70)
    
    for collection_name in ["Sentinel-1 GRD", "Sentinel-2 L2A", "Sentinel-3 LST"]:
        if collection_name in region_data:
            monthly = region_data[collection_name]
            
            if len(monthly) > 0:
                total_scenes = monthly.sum()
                avg_per_month = monthly.mean()
                max_month = monthly.idxmax()
                max_scenes = monthly.max()
                min_scenes = monthly.min()
                
                print(f"\n  {collection_name}:")
                print(f"    Total scenes: {total_scenes}")
                print(f"    Average per month: {avg_per_month:.1f}")
                print(f"    Max scenes in a month: {max_scenes} ({max_month})")
                print(f"    Min scenes in a month: {min_scenes}")
                print(f"    Months with data: {len(monthly)}")
            else:
                print(f"\n  {collection_name}:")
                print(f"    No data available")

In [ ]:
# Create a comparison bar chart showing total scenes by mission and region
fig, ax = plt.subplots(figsize=(12, 6))

regions = list(all_data.keys())
missions = ["Sentinel-1 GRD", "Sentinel-2 L2A", "Sentinel-3 LST"]

# Prepare data
data_for_plot = {mission: [] for mission in missions}

for region_name in regions:
    region_data = all_data[region_name]
    for mission in missions:
        if mission in region_data and len(region_data[mission]) > 0:
            data_for_plot[mission].append(region_data[mission].sum())
        else:
            data_for_plot[mission].append(0)

# Create grouped bar chart
x = np.arange(len(regions))
width = 0.25

colors = ['#3498db', '#2ecc71', '#e67e22']

for i, (mission, color) in enumerate(zip(missions, colors)):
    offset = width * (i - 1)
    ax.bar(x + offset, data_for_plot[mission], width, 
           label=mission, color=color, alpha=0.8)

ax.set_xlabel('Region', fontsize=12, fontweight='bold')
ax.set_ylabel('Total Number of Scenes', fontsize=12, fontweight='bold')
ax.set_title('Total Data Availability by Sentinel Mission and Region', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xticks(x)
ax.set_xticklabels(regions, fontsize=10)
ax.legend(fontsize=11, framealpha=0.9)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✓ Heatmap analysis complete!")